In [1]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

21/12/08 16:04:22 WARN Utils: Your hostname, Fans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.115 instead (on interface en0)
21/12/08 16:04:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/08 16:04:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/08 16:04:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Using “filtered_registered_business_sf.tsv” and “supervisor_sf.tsv”, list unique business names without a supervisor ordered by name.

In [3]:
supervisor_sf_pair.take(2)

[('94102', '8'), ('94102', '6')]

In [4]:
zip_biz_pair.take(1)

[('94123', ['Tournahu George L', '3301 Broderick St', 'San Francisco', 'CA'])]

In [9]:
zip_biz_pair.subtractByKey(zip_biz_pair.join(supervisor_sf_pair)).map(lambda x: x[1][0]).distinct().sortBy(lambda x: x).take(5)

['"10up, Inc."',
 '"11 Main, Inc"',
 '"1490 Chestnut Street Associates, Llc"',
 '"1946 Washington Street, Llc"',
 '"360training.com, Inc."']

In [3]:
supervisor_sf_pair.take(5)

[('94102', '8'),
 ('94102', '6'),
 ('94102', '3'),
 ('94102', '5'),
 ('94103', '8')]

In [4]:
zip_biz_pair.take(5)

[('94123', ['Tournahu George L', '3301 Broderick St', 'San Francisco', 'CA']),
 ('94124',
  ['Stephens Institute Inc', '2225 Jerrold Ave', 'San Francisco', 'CA']),
 ('94105',
  ['Stephens Institute Inc', '180 New Montgomery St', 'San Francisco', 'CA']),
 ('94108', ['Stephens Institute Inc', '540 Powell St', 'San Francisco', 'CA']),
 ('94107',
  ['Stephens Institute Inc', '460 Townsend St', 'San Francisco', 'CA'])]

In [14]:
zip_biz_pair.subtractByKey(supervisor_sf_pair).values().map(lambda x: x[0]).distinct().sortBy(lambda x: x).take(5)

['"10up, Inc."',
 '"11 Main, Inc"',
 '"1490 Chestnut Street Associates, Llc"',
 '"1946 Washington Street, Llc"',
 '"360training.com, Inc."']

## 2. Create pairs of business name and supervisor id ordered by supervisor id (first) and business name (second).

In [10]:
zip_biz_pair.join(supervisor_sf_pair).take(1)

[('94102',
  (['Stephens Institute Inc', '620 Sutter St', 'San Francisco', 'CA'], '8'))]

In [11]:
zip_biz_pair.join(supervisor_sf_pair).map(lambda x: (x[1][0][0],x[1][1])).sortBy(lambda x: (x[1],x[0])).take(5)

[('"1125 Sir Francis Drake Boulevard Operating Company, Llc"', '1'),
 ('"1909 Fillmore, Inc"', '1'),
 ('"2700 Geary Partners, L.p."', '1'),
 ('"2705 Jackson Street, Dba"', '1'),
 ('"2bkco, Inc."', '1')]

In [19]:
zip_biz_pair.join(supervisor_sf_pair).values().map(lambda x: (x[0][0],x[1])).sortBy(lambda x: (x[-1],x[0])).take(5)

[('"1125 Sir Francis Drake Boulevard Operating Company, Llc"', '1'),
 ('"1909 Fillmore, Inc"', '1'),
 ('"2700 Geary Partners, L.p."', '1'),
 ('"2705 Jackson Street, Dba"', '1'),
 ('"2bkco, Inc."', '1')]

## 3. Create pairs of business and supervisor id for all the business ordered by supervisor id (first) and business name (second). (although it may not have a district supervisor).

In [18]:
zip_biz_pair.leftOuterJoin(supervisor_sf_pair).map(lambda x: (x[1][0][0],x[1][1])).sortBy(lambda x: x[1] if x[1] else x[0]).take(5)

[('"10up, Inc."', None),
 ('"11 Main, Inc"', None),
 ('"1490 Chestnut Street Associates, Llc"', None),
 ('"1946 Washington Street, Llc"', None),
 ('"360training.com, Inc."', None)]

In [30]:
zip_biz_pair.leftOuterJoin(supervisor_sf_pair).values().map(lambda x: (x[0][0],x[1])).sortBy(lambda x: (x[-1],x[0]) if x[-1] else (x[0],)).take(10)

[('"10up, Inc."', None),
 ('"11 Main, Inc"', None),
 ('"1490 Chestnut Street Associates, Llc"', None),
 ('"1946 Washington Street, Llc"', None),
 ('"360training.com, Inc."', None),
 ('"5 Bars, Llc"', None),
 ('"5 Bars, Llc"', None),
 ('"750 Brannan Street Properties, Llc"', None),
 ('"7x7 Film, Llc"', None),
 ('"8x8, Inc."', None)]

In [31]:
zip_biz_pair.leftOuterJoin(supervisor_sf_pair).values().map(lambda x: (x[0][0],x[1]) if x[1] else (x[0][0],'')).sortBy(lambda x: (x[-1],x[0])).take(10)

[('"10up, Inc."', ''),
 ('"11 Main, Inc"', ''),
 ('"1490 Chestnut Street Associates, Llc"', ''),
 ('"1946 Washington Street, Llc"', ''),
 ('"360training.com, Inc."', ''),
 ('"5 Bars, Llc"', ''),
 ('"5 Bars, Llc"', ''),
 ('"750 Brannan Street Properties, Llc"', ''),
 ('"7x7 Film, Llc"', ''),
 ('"8x8, Inc."', '')]

In [19]:
sc.stop()